In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [3]:
data_path = "netflix_titles.csv"
df = pd.read_csv(data_path)
description = df['description']
listed_in =df["listed_in"]

In [4]:
import numpy as np
listed_in_arr = [ category.strip().lower().split(",") for category in listed_in]
dist_listed = []
temp =set()

listed_in_new = []
for category in listed_in_arr:
    temp_list = []
    for sub_cat in category:
        temp.add(sub_cat.strip())
        temp_list.append(sub_cat.strip())
    listed_in_new.append(temp_list)
dist_listed = list(temp)


print(len(dist_listed))
print(dist_listed)

42
['tv action & adventure', 'teen tv shows', 'cult movies', 'tv comedies', 'horror movies', 'tv thrillers', 'crime tv shows', 'lgbtq movies', 'stand-up comedy & talk shows', 'comedies', 'action & adventure', 'romantic movies', 'docuseries', 'stand-up comedy', 'music & musicals', 'tv dramas', 'thrillers', 'sports movies', 'anime series', 'korean tv shows', 'children & family movies', 'tv shows', 'international movies', 'classic & cult tv', 'tv mysteries', 'tv sci-fi & fantasy', 'tv horror', "kids' tv", 'british tv shows', 'anime features', 'romantic tv shows', 'classic movies', 'independent movies', 'movies', 'documentaries', 'spanish-language tv shows', 'faith & spirituality', 'reality tv', 'international tv shows', 'science & nature tv', 'dramas', 'sci-fi & fantasy']


In [5]:
print(len(listed_in_new))
cat_list_newArr  =[]
for cat_list in listed_in_new:
    temp_arr = []
    for cur_onehot in dist_listed:
        issame = False
        
        for sub_cat_list in cat_list:
            if cur_onehot==sub_cat_list:
                issame=True
                
        if issame:
            temp_arr.append(1)
        else:
            temp_arr.append(0)
    cat_list_newArr.append(np.array(temp_arr))

len(cat_list_newArr[2])

8807


42

In [6]:
num_of_flags = len(cat_list_newArr[0])
max_length = max([len(tokenizer.encode(title.strip())) for title in description])
print("max_length : ",max_length)

max_length :  62


In [7]:
len(description)

8807

#### Tensor Concat

In [8]:
import torch
from torch import tensor
X = torch.tensor([5, 5, 5])
Y = torch.tensor([6, 6, 6])
XY = torch.cat((X, Y), 0)
YX = torch.cat((Y, X), 0)
print('The tensor of XY After Concatenation:', XY)
print('The tensor of YX After Concatenation:', YX)

The tensor of XY After Concatenation: tensor([5, 5, 5, 6, 6, 6])
The tensor of YX After Concatenation: tensor([6, 6, 6, 5, 5, 5])


In [9]:
min_amount =1000
max_amount1= 1100

train_description, test_description = description[:min_amount],description[min_amount:max_amount1]
train_flags , test_flags = cat_list_newArr[:min_amount],cat_list_newArr[min_amount:max_amount1]


In [10]:
i = 0
X = torch.tensor( train_flags[i],dtype=torch.int32)
encodings_dict = tokenizer('<|startoftext|>' + train_description[i] + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
Y =  torch.tensor(encodings_dict['input_ids'],dtype=torch.int32)
print(X.shape)
print(Y.shape)                                  
XY = torch.cat((Y,X), 0)
print(XY.shape)
print(XY)

torch.Size([42])
torch.Size([62])
torch.Size([104])
tensor([50257,  1722,   607,  2988,  1474,    82,   262,   886,   286,   465,
         1204,    11, 26479, 39700,   268,  5030,  9539,   465,  1918,   287,
        47602,   290,   401,   605,  2842,   284,  1037,   606,  1111,  1986,
          262, 13203,    13, 50256, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     1,     0,     0,     0,
            0,     0,     0,     0], dtype=torch.int32)


In [11]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list,flags, tokenizer, max_length,batch_size):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        self.batch_size = batch_size

        for txt,cur_flag in zip(txt_list,flags):
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
            input_ids_X  = torch.tensor(encodings_dict['input_ids'])
            cur_flag_tensor = torch.tensor(cur_flag)
            input_ids_X_cur_flag = torch.cat((input_ids_X,cur_flag_tensor), 0)
            self.input_ids.append(input_ids_X_cur_flag)


            attention_mask_X  = torch.tensor(encodings_dict['attention_mask'])
            attention_mask_X_cur_flag = torch.cat((attention_mask_X,cur_flag_tensor), 0)
            self.attn_masks.append(attention_mask_X_cur_flag)

    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, idx):
        #input_ids_X_batches = self.input_ids[idx*self.batch_size : (idx+1)*self.batch_size]
        #attn_masks_X_batches = self.attn_masks[idx*self.batch_size : (idx+1)*self.batch_size]
        
        return self.input_ids, self.attn_masks

In [12]:
dataset = NetflixDataset(train_description,train_flags, tokenizer, max_length=max_length,batch_size=8)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [13]:

for temp in train_dataset:
    print(temp[0][0])
    print(temp[1][0])
    data = torch.stack([f[0] for f in temp])
    print(data.shape)
    data = torch.stack([f[1][0] for f in temp])
    print(data[0])
    break
    """
    if len(temp[0])!=len(temp[1]) and len(temp[0])==62:
        print(temp[0])
        print(temp[1])
    
        break
    """


tensor([50257,  1722,   607,  2988,  1474,    82,   262,   886,   286,   465,
         1204,    11, 26479, 39700,   268,  5030,  9539,   465,  1918,   287,
        47602,   290,   401,   605,  2842,   284,  1037,   606,  1111,  1986,
          262, 13203,    13, 50256, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     1,     0,     0,     0,
            0,     0,     0,     0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 

In [14]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=100, save_steps=500,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [15]:
next(model.parameters()).is_cuda

True

In [16]:
Trainer(model=model, args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0][0] for f in data]),
                                                              'attention_mask': torch.stack([f[1][0] for f in data]),
                                                              'labels': torch.stack([f[0][0] for f in data])}).train()

c:\Users\tharh\anaconda3\envs\tharhtet\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 900
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 900


  0%|          | 0/900 [00:00<?, ?it/s]

{'loss': 3.5685, 'learning_rate': 4.4943820224719104e-05, 'epoch': 0.11}
{'loss': 0.0744, 'learning_rate': 3.9325842696629214e-05, 'epoch': 0.22}
{'loss': 0.0304, 'learning_rate': 3.370786516853933e-05, 'epoch': 0.33}
{'loss': 0.0092, 'learning_rate': 2.8089887640449443e-05, 'epoch': 0.44}


Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


{'loss': 0.0091, 'learning_rate': 2.2471910112359552e-05, 'epoch': 0.56}


Model weights saved in ./results\checkpoint-500\pytorch_model.bin


{'loss': 0.0226, 'learning_rate': 1.6853932584269665e-05, 'epoch': 0.67}
{'loss': 0.0015, 'learning_rate': 1.1235955056179776e-05, 'epoch': 0.78}
{'loss': 0.0015, 'learning_rate': 5.617977528089888e-06, 'epoch': 0.89}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 0.0065, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 55.6422, 'train_samples_per_second': 16.175, 'train_steps_per_second': 16.175, 'train_loss': 0.4137448890341653, 'epoch': 1.0}


TrainOutput(global_step=900, training_loss=0.4137448890341653, metrics={'train_runtime': 55.6422, 'train_samples_per_second': 16.175, 'train_steps_per_second': 16.175, 'train_loss': 0.4137448890341653, 'epoch': 1.0})

### Sampel prediction

In [17]:
inputS = torch.stack([f[0][0] for f in val_dataset])
print(inputS[0])
print(len(inputS[0]))


tensor([50257,  1722,   607,  2988,  1474,    82,   262,   886,   286,   465,
         1204,    11, 26479, 39700,   268,  5030,  9539,   465,  1918,   287,
        47602,   290,   401,   605,  2842,   284,  1037,   606,  1111,  1986,
          262, 13203,    13, 50256, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     1,     0,     0,     0,
            0,     0,     0,     0])
104


In [18]:
i = 0
X = torch.tensor( train_flags[i])
encodings_dict = tokenizer('<|startoftext|>' + train_description[i] + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
Y =  torch.tensor(encodings_dict['input_ids'])
#print(X.shape)
#print(Y.shape)                                  
XY = torch.cat((Y,X), 0)

sample_input = torch.reshape(XY,(1,-1)).cuda()
#print(sample_input)
#print(sample_input[0].shape)
#print(len(sample_input[0]))

sample_outputs = model.generate(sample_input,no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2,max_new_tokens=50)
#print(len(sample_outputs))
print("Input : ",train_description[i])
print("Generated output : ",tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input :  As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.
Generated output :  As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"!!!!!!!!" (!) "Avengers: My Father's Bawling & Mother Of Theoms For Our Fathers As People Were In Troubles To Begging Their Ownings And They Still Boarding With A Differently Even though they all love their own


In [19]:
train_flags[i]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [20]:
print(dist_listed)
print(len(dist_listed))

['tv action & adventure', 'teen tv shows', 'cult movies', 'tv comedies', 'horror movies', 'tv thrillers', 'crime tv shows', 'lgbtq movies', 'stand-up comedy & talk shows', 'comedies', 'action & adventure', 'romantic movies', 'docuseries', 'stand-up comedy', 'music & musicals', 'tv dramas', 'thrillers', 'sports movies', 'anime series', 'korean tv shows', 'children & family movies', 'tv shows', 'international movies', 'classic & cult tv', 'tv mysteries', 'tv sci-fi & fantasy', 'tv horror', "kids' tv", 'british tv shows', 'anime features', 'romantic tv shows', 'classic movies', 'independent movies', 'movies', 'documentaries', 'spanish-language tv shows', 'faith & spirituality', 'reality tv', 'international tv shows', 'science & nature tv', 'dramas', 'sci-fi & fantasy']
42


In [21]:
input_text = "Scientists conduct research on sharks"



input_flags = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]




for flag,cat in zip(input_flags,dist_listed):
    print(str(flag)+"\t:"+cat)

0	:tv action & adventure
0	:teen tv shows
0	:cult movies
0	:tv comedies
0	:horror movies
0	:tv thrillers
0	:crime tv shows
0	:lgbtq movies
0	:stand-up comedy & talk shows
0	:comedies
0	:action & adventure
0	:romantic movies
0	:docuseries
0	:stand-up comedy
0	:music & musicals
0	:tv dramas
0	:thrillers
0	:sports movies
0	:anime series
1	:korean tv shows
0	:children & family movies
0	:tv shows
0	:international movies
0	:classic & cult tv
0	:tv mysteries
0	:tv sci-fi & fantasy
1	:tv horror
0	:kids' tv
0	:british tv shows
0	:anime features
0	:romantic tv shows
0	:classic movies
0	:independent movies
0	:movies
0	:documentaries
0	:spanish-language tv shows
0	:faith & spirituality
0	:reality tv
0	:international tv shows
0	:science & nature tv
0	:dramas
0	:sci-fi & fantasy


In [22]:
i = 0
X = torch.tensor(input_flags)


encodings_dict = tokenizer('<|startoftext|>' + input_text + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
Y =  torch.tensor(encodings_dict['input_ids'])    

print("X ",tokenizer.decode(X, skip_special_tokens=True))
XY = torch.cat((Y,X), 0)
sample_input = torch.reshape(XY,(1,-1)).cuda()

sample_outputs = model.generate(sample_input,no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2,max_new_tokens=50)
print("Input : ",input_text)


print("Generated output : ",tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


X  !!!!!!!!!!!!!!!!!!!"!!!!!!"!!!!!!!!!!!!!!!
Input :  Scientists conduct research on sharks
Generated output :  Scientists conduct research on sharks!!!!!!!!!!!!!!!!!!!"!!!!!!"!!!!!!!!!!!!!!!(s) (As her father dies in the end of his life to help them both face their paths together as they go along Their parents battle against other people's cruel and anger towards themselves - all eyes will soon be greeted with a sickeningly


In [23]:
Input :  stranger
Generated output :  stranger!!!!!!!!!!!!!!!!!!!!!!"!!!!!!!!!!!!!!!!!!!!'
As her grows his father nears, they come to help them both face the wrath of their captors and provide themselves with a great chance at its own peril. And there will be no one who has everything in my heart as it comes

SyntaxError: invalid syntax (2534928701.py, line 2)